<a href="https://colab.research.google.com/github/alexmackfi/Time_Series_2024/blob/main/ReportForExam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sktime==0.31.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 kB 11.3 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError

from sktime.forecasting.ets import AutoETS
from prophet import Prophet

import warnings
warnings.filterwarnings("ignore")
SEED = 2024

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# Экзаменационное задание по временным рядам: Прогнозирование продаж в магазинах офлаин-ритейлера в США <a class="tocSkip">

В качестве экзаменационного задания вам предстоит построить модель прогнозирования спроса(продаж) на товары в магазинах офлаин-ритейлера в США . Всего в датасете 3 магазина, в каждом магазине по 15 артикулов (товаров). Вам нужно выбрать 1 магазин (любой из 3).

Прогнозировать продажи нужно на неделю, на месяц и на квартал.

В качестве дополнительной информации вам переданы данные о цене товара (меняются раз в неделю, а также о праздниках в США).

Задание 1. Реализовать класс, который умеет:

1) предобрабатывать исходные данные в удобный формат;

2) обучаться для задачи прогнозирования;

3) оценивать качество своих прогнозов;

4) сохранять модели и подгружать их;

5) Прогнозировать продажи на неделю, на месяц и на квартал.

Должна быть рабочая программа, которая делает инференс (прогнозирование на произвольном тестовом датасете (аналогично тому, который есть у вас)).


Задача 2. Подготовить отчёт о решении данной задачи в виде jupyter ноутбука. В отчёте, в частности, вы должны ответить на следующие вопросы:

1) Какие методы предобработки данных вы использовали?

2) Какие модели пробовали? Почему пробовали именно их?

3) Как вы проверяете качество модели? На каких данных? Какие метрики используюте? Чем обусловлен выбор именно этих метрик?

4) Какое итоговое качество модели на тестовом датасете?

# Задание 1

In [3]:
class DataTransformer:
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, sales_df, prices_df, calendar_df):

        # получение данных для обучения
        day_data = pd.merge(sales_df,calendar_df,how='left',on = 'date_id')
        day_data = day_data.drop(columns=['weekday','event_name_1', 'event_type_1',
                                          'event_name_2', 'event_type_2', 'CASHBACK_STORE_1',
                                          'CASHBACK_STORE_2','CASHBACK_STORE_3'])
        day_data = pd.merge(day_data, prices_df, how='left', on = ['item_id',	'wm_yr_wk'])
        day_data["date"] = pd.to_datetime(day_data["date"])
        for id in day_data['item_id'].value_counts().index:
            day_data[day_data['item_id']==id] = day_data[day_data['item_id']==id].ffill().bfill()

        # получение календаря с праздниками
        calendar = calendar_df.copy().set_index('date')
        calendar.index = pd.to_datetime(calendar.index, format='%Y-%m-%d')
        calendar.sort_index(inplace=True)
        calendar_cashback_col = calendar['CASHBACK_STORE_2']
        calendar.drop(['CASHBACK_STORE_1', 'CASHBACK_STORE_2', 'CASHBACK_STORE_3', 'wday', 'month', 'year', 'weekday'],
                      axis=1,
                      inplace=True)
        fill_values = {'event_name_1': 'noevent',
                       'event_name_2': 'noevent',
                       'event_type_1': 'notype',
                       'event_type_2': 'notype'
        }
        calendar.fillna(value = fill_values, inplace=True)
        calendar['CASHBACK_STORE_2'] = calendar_cashback_col
        holidays = calendar.drop(['event_type_1','event_type_2','wm_yr_wk','date_id'],axis=1)

        conditions = [
        (holidays.event_name_1 != 'noevent') & (holidays.event_name_2 != 'noevent'),
        (holidays.event_name_1 != 'noevent') ^ (holidays.event_name_2 != 'noevent'),
        (holidays.event_name_1 == 'noevent') & (holidays.event_name_2 == 'noevent')
        ]

        lower_window = [-5, -3, None]
        upper_window = [5, 3, None]

        holidays['lower_window'] = np.select(conditions, lower_window)
        holidays['upper_window'] = np.select(conditions, upper_window)

        holidays = holidays.dropna()
        holidays[holidays.event_name_1 == 'noevent']['event_name_1'] = holidays[holidays.event_name_1 == 'noevent']['event_name_2']
        holidays['holiday'] = holidays['event_name_1']
        holidays = holidays.drop(['event_name_1','event_name_2'],axis=1)
        holidays['ds']=holidays.index

        return day_data, holidays

In [22]:
class SolverModel:

    def __init__(self, soles, prices, calendar, data_transformer = DataTransformer()):

        self.data = data_transformer.transform(soles, prices, calendar)[0]
        self.holidays = data_transformer.transform(soles, prices, calendar)[1]
        self.briefly_models = [AutoETS(auto=True, sp=7, n_jobs=-1) for i in enumerate(self.data.item_id.unique())]
        self.longterm_models = [Prophet(holidays=self.holidays) for i in enumerate(self.data.item_id.unique())]

    def fit(self):

        for i, art in enumerate(self.data.item_id.unique()):
            data_train = self.data[self.data['item_id']==art][['date','cnt']].set_index('date')
            self.briefly_models[i].fit(data_train)

            data_train['ds']=data_train.index
            data_train['y']=data_train['cnt']
            self.longterm_models[i].fit(data_train)
        # self.briefly_models = joblib.load('model_ets.joblib')
        # self.longterm_models = joblib.load('model_prophet.joblib')

        return self

    def predict(self, horizon = 'W'):

        horizon_dict = {'W':7,'M':30,'Q':90}
        horizon = horizon_dict[horizon]
        predicts = {}

        if horizon <=30:
            for i, art in enumerate(self.data.item_id.unique()):
                data_test = self.data[self.data['item_id']==art][['date','cnt']].set_index('date').iloc[-90:]
                predicts[art] = self.briefly_models[i].predict(data_test.iloc[:horizon].index)

        else:
            for i, art in enumerate(self.data.item_id.unique()):
                future = self.longterm_models[i].make_future_dataframe(periods=horizon)
                forecast = self.longterm_models[i].predict(future)
                predicts[art] = forecast['yhat'].iloc[-horizon:]


        preds = pd.concat(predicts, axis=1).reindex(predicts['STORE_1_064'].index)
        return preds

    def evaluate(self):

        mae_func = MeanAbsoluteError()
        mse_func = MeanSquaredError()
        stats_list = ['mean']

        res_w = self. predict('W')
        res_m = self. predict('M')
        res_q = self. predict('Q')

        result7,result30,result90 = [],[],[]

        for i, art in enumerate(self.data.item_id.unique()):

            data_test = self.data[self.data['item_id']==art][['date','cnt']].set_index('date').iloc[-90:]

            y_true_w = data_test.iloc[:7]
            y_pred_w = res_w[art]
            y_pred_w.index = y_true_w.index

            maes = mae_func.evaluate_by_index(y_true_w,y_pred_w)
            mses = mse_func.evaluate_by_index(y_true_w,y_pred_w)

            result7.append(pd.DataFrame([maes.agg(stats_list),
                                mses.agg(stats_list),],
                            index=['MAE','MSE']))

            y_true_m = data_test.iloc[:30]
            y_pred_m = res_m[art]
            y_pred_m.index = y_true_m.index

            maes = mae_func.evaluate_by_index(y_true_m,y_pred_m)
            mses = mse_func.evaluate_by_index(y_true_m,y_pred_m)

            result30.append(pd.DataFrame([maes.agg(stats_list),
                                mses.agg(stats_list),],
                            index=['MAE','MSE']))

            y_true_q = data_test
            y_pred_q = res_q[art]
            y_pred_q.index = y_true_q.index

            maes = mae_func.evaluate_by_index(y_true_q,y_pred_q)
            mses = mse_func.evaluate_by_index(y_true_q,y_pred_q)

            result90.append(pd.DataFrame([maes.agg(stats_list),
                                mses.agg(stats_list),],
                            index=['MAE','MSE']))

        result =pd.concat([pd.concat(result7,axis=1).mean(axis=1),
                         pd.concat(result30,axis=1).mean(axis=1),
                         pd.concat(result90,axis=1).mean(axis=1)],
                         axis=1)
        result.columns = ['Week','Month','Quarter']
        return result

    def save_model(self, filepath):
        joblib.dump((self.briefly_models,self.longterm_models),filepath)
        return self

    def load_model(self, filepath):
        self.briefly_models,self.longterm_models = joblib.load(filepath)
        return self

# Задание 1. Проверка

## Посмотрим на тестовые данные

In [5]:
sales = pd.read_csv('exam_data/shop_sales_test.csv')
sales = sales[sales.store_id=='STORE_1'].drop(columns=['store_id'])
sales.tail()

,item_id,date_id,cnt
4225,STORE_1_727,1909,1
4226,STORE_1_727,1910,1
4227,STORE_1_727,1911,4
4228,STORE_1_727,1912,1
4229,STORE_1_727,1913,5


In [6]:
prices = pd.read_csv('exam_data/shop_sales_prices_test.csv')
prices = prices[prices.store_id=='STORE_1'].drop(columns=['store_id'])
prices.tail()

,item_id,wm_yr_wk,sell_price
1030,STORE_1_584,11621,1.00
1031,STORE_1_586,11621,1.68
1032,STORE_1_587,11621,2.48
1033,STORE_1_714,11621,1.58
1034,STORE_1_727,11621,3.98


In [7]:
calendar = pd.read_csv('exam_data/shop_sales_dates_test.csv')
calendar.tail()

,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,date_id,CASHBACK_STORE_1,CASHBACK_STORE_2,CASHBACK_STORE_3
145,2016-06-15,11620,Wednesday,5,6,2016,NaN,NaN,NaN,NaN,1965,1,0,1
146,2016-06-16,11620,Thursday,6,6,2016,NaN,NaN,NaN,NaN,1966,0,0,0
147,2016-06-17,11620,Friday,7,6,2016,NaN,NaN,NaN,NaN,1967,0,0,0
148,2016-06-18,11621,Saturday,1,6,2016,NaN,NaN,NaN,NaN,1968,0,0,0
149,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,1969,0,0,0


In [8]:
transformer = DataTransformer()
df, holidays = transformer.transform(sales,prices,calendar)
df.dtypes

,0
item_id,object
date_id,int64
cnt,int64
date,datetime64[ns]
wm_yr_wk,int64
wday,int64
month,int64
year,int64
sell_price,float64


In [9]:
holidays

,CASHBACK_STORE_2,lower_window,upper_window,holiday,ds
date,,,,,
2016-02-07,1,-3,3,SuperBowl,2016-02-07
2016-02-10,1,-3,3,LentStart,2016-02-10
2016-02-14,0,-3,3,ValentinesDay,2016-02-14
2016-02-15,0,-3,3,PresidentsDay,2016-02-15
2016-02-17,0,-3,3,LentWeek2,2016-02-17
2016-03-17,0,-3,3,StPatricksDay,2016-03-17
2016-03-24,0,-3,3,Purim End,2016-03-24
2016-03-27,0,-3,3,Easter,2016-03-27
2016-04-30,0,-3,3,Pesach End,2016-04-30


## Обучение на тестовых данных

In [10]:
ob = SolverModel(sales,prices,calendar)
ob.fit()

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp13cq2on7/_so10ofz.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp13cq2on7/45j0zwp4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=8984', 'data', 'file=/tmp/tmp13cq2on7/_so10ofz.json', 'init=/tmp/tmp13cq2on7/45j0zwp4.json', 'output', 'file=/tmp/tmp13cq2on7/prophet_modelo6ozbvdc/prophet_model-20241003072320.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
07:23:20 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
07:23:23 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling yearly seasonalit

## Получение прогноза

In [11]:
ob.predict('W')

,STORE_1_064,STORE_1_065,STORE_1_325,STORE_1_376,STORE_1_090,STORE_1_252,STORE_1_339,STORE_1_546,STORE_1_547,STORE_1_555,STORE_1_584,STORE_1_586,STORE_1_587,STORE_1_714,STORE_1_727
,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt
date,,,,,,,,,,,,,,,
2016-01-26,0.202047,0.113864,6.254605,0.361858,51.357500,5.470749,4.734844,1.576218,14.010388,12.707663,1.500201,13.651628,20.163700,18.425492,2.691728
2016-01-27,0.202026,0.093532,6.254179,0.361922,52.012554,5.268422,4.734470,2.414357,14.009087,11.823624,1.500151,17.653011,27.310623,14.966335,2.153046
2016-01-28,0.202006,0.076831,6.253654,0.361885,59.450839,7.084955,4.734097,1.583174,14.008386,15.193279,1.500001,17.461061,26.711882,18.391579,2.076061
2016-01-29,0.201986,0.241675,6.253729,0.361849,93.932137,12.087397,4.733623,2.059344,14.008285,18.979596,1.500251,24.245427,28.951776,21.774819,3.570643
2016-01-30,0.201966,0.198521,6.253803,0.361813,130.033980,20.643281,4.733550,3.465637,14.007385,30.689005,1.500101,33.575531,44.174735,23.886695,4.856491
2016-01-31,0.201946,0.163072,6.253378,0.361877,113.714562,19.565264,4.733977,3.233886,14.008884,25.341190,1.500051,29.451819,52.202662,17.108718,7.285195
2016-02-01,0.201925,0.133953,6.253553,0.361841,67.941818,13.090146,4.733903,2.606256,14.009183,21.178930,1.500101,21.620712,44.128482,14.546122,2.307339


In [12]:
ob.predict('M')

,STORE_1_064,STORE_1_065,STORE_1_325,STORE_1_376,STORE_1_090,STORE_1_252,STORE_1_339,STORE_1_546,STORE_1_547,STORE_1_555,STORE_1_584,STORE_1_586,STORE_1_587,STORE_1_714,STORE_1_727
,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt,cnt
date,,,,,,,,,,,,,,,
2016-01-26,0.202047,0.113864,6.254605,0.361858,51.357500,5.470749,4.734844,1.576218,14.010388,12.707663,1.500201,13.651628,20.163700,18.425492,2.691728
2016-01-27,0.202026,0.093532,6.254179,0.361922,52.012554,5.268422,4.734470,2.414357,14.009087,11.823624,1.500151,17.653011,27.310623,14.966335,2.153046
2016-01-28,0.202006,0.076831,6.253654,0.361885,59.450839,7.084955,4.734097,1.583174,14.008386,15.193279,1.500001,17.461061,26.711882,18.391579,2.076061
2016-01-29,0.201986,0.241675,6.253729,0.361849,93.932137,12.087397,4.733623,2.059344,14.008285,18.979596,1.500251,24.245427,28.951776,21.774819,3.570643
2016-01-30,0.201966,0.198521,6.253803,0.361813,130.033980,20.643281,4.733550,3.465637,14.007385,30.689005,1.500101,33.575531,44.174735,23.886695,4.856491
2016-01-31,0.201946,0.163072,6.253378,0.361877,113.714562,19.565264,4.733977,3.233886,14.008884,25.341190,1.500051,29.451819,52.202662,17.108718,7.285195
2016-02-01,0.201925,0.133953,6.253553,0.361841,67.941818,13.090146,4.733903,2.606256,14.009183,21.178930,1.500101,21.620712,44.128482,14.546122,2.307339
2016-02-02,0.201905,0.110034,6.253327,0.361804,51.354722,9.269875,4.734230,1.393776,14.010282,18.001652,1.500051,17.707940,39.610526,18.808391,2.691565


In [13]:
ob.predict('Q')

,STORE_1_064,STORE_1_065,STORE_1_325,STORE_1_376,STORE_1_090,STORE_1_252,STORE_1_339,STORE_1_546,STORE_1_547,STORE_1_555,STORE_1_584,STORE_1_586,STORE_1_587,STORE_1_714,STORE_1_727
94,0.010374,0.265044,4.300843,0.553942,65.287804,19.856729,6.271499,0.626646,11.166014,31.076563,1.276423,32.102144,31.852538,18.667347,1.938880
95,0.260374,0.639221,6.550530,0.178899,51.787782,17.982413,1.647258,0.093440,7.918834,26.826915,1.525393,26.429794,22.482253,19.787910,2.813756
96,0.010488,0.389955,6.675349,0.303948,53.037780,16.982577,3.271699,-0.064488,15.785462,24.202050,1.149665,31.382647,25.980764,17.665378,1.688883
97,0.383727,0.618846,7.983792,0.546165,63.364338,15.451274,2.180060,0.450551,13.896924,26.621904,1.790576,32.216143,20.427280,20.208800,2.322695
98,0.454556,0.802874,7.836567,0.164457,99.650587,20.724328,3.775042,-0.504796,3.831088,25.620538,0.903602,33.959475,22.298481,27.861712,3.273064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,0.278709,0.883574,8.825731,0.272847,56.871490,27.335071,1.249093,-4.640566,2.098168,36.776709,1.353407,34.636308,26.184720,20.722710,2.706437
180,0.028823,0.634309,8.950550,0.397896,58.121488,26.335235,2.873534,-4.798495,9.964796,34.151843,0.977679,39.589160,29.683231,18.600178,1.581564
181,0.402062,0.863199,10.258992,0.640113,68.448045,24.803931,1.781896,-4.283455,8.076258,36.571698,1.618590,40.422657,24.129746,21.143600,2.215376
182,0.472891,1.047227,10.111768,0.258406,104.734295,30.076985,3.376877,-5.238802,-1.989578,35.570332,0.731616,42.165989,26.000948,28.796512,3.165746


## Оценка

In [14]:
ob.evaluate()

,Week,Month,Quarter
MAE,3.619187,4.333394,7.128966
MSE,37.282135,57.460342,152.284487


## Сохранение моделей

In [15]:
ob.save_model('test_model.joblib')

## Подгрузка моделей

In [16]:
ob.load_model('test_model.joblib')

# Задание 2

## 1) Какие методы предобработки данных вы использовали?

**Для предобработки данных ипользовались функции и методы `pandas`:**

- для заполнения пропусков - методы `ffill()` и `bfill()`  в комбинации

- для преобразованиея дат в нужный формат - функция `pd.to_datetime`

- для оценок и анализа данных - различные статистические методы (`corr`, `var`, `describe`) и разные методы визуализаций

- для создания календаря праздников - метод `dropna` для удаления дат без праздников и дополнительно не из `pandas` - логические операции (`&` , `^` , `|`) и функция из `numpy` - `np.select`


## 2) Какие модели пробовали? Почему пробовали именно их?


**Для этой работы я пробовала:**

- **Наивный прогноз** в качестве бейзлайна, чтобы иметь стартовые метрики. Мой базовый прогноз - это предыдущие значения соответствующего диапозона (предыдущая неделя/месяц/квартал)

> `Не выбран по понятным причинам`

- **ARIMA**, как модель, хорошо улавливающая краткосрочные закономерности в данных, её почти всегда пробуют в первую очередь

> `Не выбрана, так как проиграла другому прогнозу (хоть и была в числе финалистов с одними из лучших метрик на краткосрок)`

- **Тройное экспоненциальное сглаживание**, как довольно простая и быстрая модель

> `Не выбрана, так как из-за маленькой дисперсии в данных прогноз превратился в усреденение`

- **AutoETS** как попытка в автоподбор параметров

> `Выбрана для краткосрочных прогнозов (на  неделю и на месяц)`

- **TBATS**, как вариант можели посложнее

> `Не выбрана, модель дала неплохие результаты, но немного проиграла по метрикам другим кандидатам при очень больших временных затратах на обучение относительно них`

- **Prophet** в контексте предположения о том, что праздники оказывают влияние на продажи товаров

> `Выбрана, для долгосрочных прогнозов (на квартал)`

## 3) Как вы проверяете качество модели? На каких данных? Какие метрики используюте? Чем обусловлен выбор именно этих метрик?




> `Важно упомянуть, что все данные упорядочены по времени`



**В момент обучения модели на тренировочных данных:**

- Качество проверялось на отложенной выборке из 90 наблюдений (квартал)

- Для оценки качества моделей при работе с предсказаниями на `неделю`/`месяц`/`квартал` соответственно из отложенной выборки брались первые `7`/`30`/`90` наблюдений

- Для оценки качества моделей на тестовых данных использовались *средние* и *мединные* прогнозы `MAE`, `MSE`, `R2-score`

> `MSE` - стандартный выбор, в данных нет больших выбросов  

> `MAE` - пусть в данных нет особо выбросов, но есть большой разброс от товара к товару. Эта метрика позволит его немного стабилизировать в контексте средней оценки

> `R2-score` - в итоге убрала из списка метрик для тестового сета, изначально хотелось посмотреть на влияние дисперсии между предсказаниями и истинными значениями

> `MAPE` - не использовалась, так как не давала интерпритируемые результаты (слишком много измерений близких к нулю)

**В момент оценки модели на тестовых данных:**

- Оценивалась на тестовой выборке почти полностью, из 94 наблюдений были взяты первые 90

- Для оценки качества моделей при работе с предсказаниями на `неделю`/`месяц`/`квартал` соответственно из выборки брались первые `7`/`30`/`90` наблюдений

- Для оценки качества моделей на тестовых данных использовались *средние* прогнозы `MAE`, `MSE`, как наиболее легко интерпритируемые метрики из тестовых



## 4) Какое итоговое качество модели на тестовом датасете?

In [21]:
ob.load_model('model.joblib')
ob.evaluate()

,Week,Month,Quarter
MAE,6.290624,4.696872,7.294909
MSE,74.939094,47.965292,125.349167
